# 1. Summary

- 数据并行 vs 模型并行
    - 数据并行：模型拷贝(per device), 数据split/chunk (批次粒度)
    - 模型并行: 数据拷贝(per device), 模型split/chunk, 单卡放不下模型

- DP -> DDP
    - DP: nn.DataParallel
    - DDP: DistributedDataParallel
    

# 2. Imports and parameters

In [8]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch import *
from torch.nn import *

In [2]:
input_size = 5
output_size = 2
batch_size = 30
data_size = 100

In [18]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda', index=0)

# 3. dummy dataset

In [4]:
from typing import Any


class RandomDataset(Dataset):

    def __init__(self, size, length):
        self.len = length

        self.data = torch.randn(length, size)

    def __getitem__(self, index):
        return self.data[index]
    
    def __len__(self):
        return self.len
    

rand_loader = DataLoader(
    dataset=RandomDataset(input_size, data_size),
    batch_size=batch_size,
    shuffle=True
)

# Simple model

In [15]:
class Model(nn.Module):

    def __init__(self, input_size, output_size):
        super(Model, self).__init__()

        self.fc = nn.Linear(input_size, output_size)

    def forward(self, input):
        output = self.fc(input)

        print("Input size: ", input_size)
        print("Output size: ", output_size)
        
        return output

# 5. DataParallel

In [19]:
model = Model(input_size, output_size)
model.to(device)
model

Model(
  (fc): Linear(in_features=5, out_features=2, bias=True)
)

In [20]:
model = Model(input_size, output_size)

if torch.cuda.device_count() >= 1:
    print("Use ", torch.cuda.device_count(), " GPUs")

    model = nn.parallel.DataParallel(model)

Use  1  GPUs


In [21]:
model

DataParallel(
  (module): Model(
    (fc): Linear(in_features=5, out_features=2, bias=True)
  )
)

In [23]:
model.to(device)

DataParallel(
  (module): Model(
    (fc): Linear(in_features=5, out_features=2, bias=True)
  )
)

## 5.1 tensor

In [25]:
a = torch.randn(3, 4)
a.is_cuda

False

In [28]:
b = a.to(device)
b.is_cuda


True

## 5.2 models

In [29]:
a = Model(3, 4)
next(a.parameters()).is_cuda

False

In [30]:
b = a.to(device)
next(b.parameters()).is_cuda

True

In [31]:
next(a.parameters()).is_cuda

True